In [1]:
from scipy.integrate import quad
from numpy import *
from matplotlib.pylab import *
import matplotlib as mpl
%matplotlib inline
#from pandas.core import *
import pandas as pd

from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics

from sklearn.decomposition import PCA,FastICA

from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.datasets.samples_generator import make_blobs
from sklearn.preprocessing import StandardScaler
import pickle


In [2]:
folder = '/mnt/data/spinemodel/conditions/'

In [3]:
#Jupyters: Proper Spine Model - Inhibition.ipynb
with open(folder+"dis_baselinev3_1000_80_65sh.pickle","rb") as f:
    vavgDiS,mesDiS,vtracsDiS,vtracsDDiS,vtracsSDiS,CtracsDiS = pickle.load(f)

with open(folder+"All_baseline_datasetv3_1000_80_65sh.pickle","rb") as f:
    vavgT,mesT,vtracsT,vtracsDT,vtracsST,CtracsT,me2T,_,dataT = pickle.load(f)


In [4]:
dataT.keys(),vtracsST.shape

(dict_keys(['#Sp', '#Bch', 'Dsb', 'Dss', 'd_interSp', 'Vtot', 'Vh', 'Ln', 'SA', 'nPSD', 'A1', 'A2', 'GPHN_SNR', 'DiS-EMchk', 'Lneck', 'maxDneck', 'minDneck', 'meanDneck', 'Rneck', 'Lhead', 'maxDhead', 'minDhead', 'meanDhead', 'Ah', 'Rhead', 'meanDneck_trunc70pc', 'L', 'D', 'AhA0', 'Rmorph']),
 (10002, 384))

In [5]:
pddataT = pd.DataFrame(dataT)

In [6]:
labels = ('Vh','A1','Ah','Dss','Rneck','Lhead','meanDhead')
datt = column_stack((dataT['Vh'],dataT['A1'],dataT['Ah'],dataT['Dss'],dataT['Rneck'],dataT['Lhead'],dataT['meanDhead']))

labels = ('A1','Dss','Rneck','SA')
datt = column_stack((dataT['A1'],dataT['Dss'],dataT['Rneck'],dataT['SA']))

In [7]:
pddataT = pd.DataFrame(data=datt,columns=labels)

In [8]:
regressor = LinearRegression()  
regressor.fit(datt, mesT[:,3])
coeforig = regressor.coef_*1.0
print(regressor.coef_)

[ 3.06790858e+00 -3.93328064e-03 -5.48712705e-05 -8.98420167e-03]


In [9]:
y_pred = regressor.predict(datt)

In [10]:
fullvarlinear = ((mesT[:,3]-y_pred)).var()
print(fullvarlinear)

0.008987657974958048


In [11]:
def LinReg_varexplain_B(X,y,labels= None, relative = False):
    if labels is None:
        try: 
            labels = X.keys()
        except:
            labels = arange(X.shape[1])
            
    idcs = arange(X.shape[1]).tolist()
    vardat = zeros((X.shape[1],2))
    
    var0 = var(y)
    #y_predt = y*0.0    
    for k in range(X.shape[1]-1):

        varts = zeros(len(idcs))
        #print(idcs)
        for iv,i in enumerate(idcs):
            
            idct = 1*array(idcs)
            idct = delete(idct,iv)
            Xb = X.values[:,array(idct)]
            
            regressor.fit(Xb, y)
            y_pred = regressor.predict(Xb)
            vart = ((y-y_pred)).var()
            
            varts[iv] = vart
        
        #print(varts)
        # We select column that affected the minimum and remove it
        sel = argmin(varts)
        i = idcs[sel]
        vardat[k,:] = (idcs[sel],varts[sel])
        idcs.pop(sel)

    #i = idcs[0]
    vardat[-1,:] = (idcs[0],var(y))
    #print(vardat,var0)
    #hist(y-y_pred,41)

    result = {}
    #print(vardat)
    for i in range(vardat.shape[0]-1,0,-1):
        vsa = (vardat[i,1])/var0
        vs = (vardat[i-1,1])/var0
        #print(labels[int(vardat[i,0])],(vsa-vs)*100)
        result[labels[int(vardat[i,0])]] = int(10000*(vsa-vs))/10000
        
    regressor.fit(X, y)
    y_pred = regressor.predict(X)
    vart = ((y-y_pred)).var()
    vsa = (vardat[0,1])/var0
    vs = (vart)/var0
    
    #print(labels[int(vardat[0,0])],(vsa-vs)*100)
    result[labels[int(vardat[0,0])]] = int(10000*(vsa-vs))/10000
    result['variance'] = int(10000*(var0))/10000
    return(result)

In [12]:
def LinReg_varexplain_F(X,y,labels= None, relative = False):
    if labels is None:
        try: 
            labels = X.keys()
        except:
            labels = arange(X.shape[1])
            
    idcs = arange(X.shape[1]).tolist()
    vardat = zeros((X.shape[1],2))
    
    y_predt = y*0.0
    var0 = var(y)
    minvar = var0
    for k in range(X.shape[1]-1):
        varts = zeros(len(idcs))
        for iv,i in enumerate(idcs):
            regressor.fit(X.values[:,i:(i+1)], y-y_predt)
            y_pred = regressor.predict(X.values[:,i:(i+1)])
            vart = ((y-y_predt-y_pred)).var()
            varts[iv] = vart
        sel = argmin(varts)
        i = idcs[sel]
        regressor.fit(X.values[:,i:(i+1)], y-y_predt)
        y_pred = regressor.predict(X.values[:,i:(i+1)])
        y_predt += y_pred

        vardat[k,:] = (idcs[sel],varts[sel])
        idcs.pop(sel)

    i = idcs[0]
    regressor.fit(X.values[:,i:(i+1)], y-y_predt)
    y_pred = regressor.predict(X.values[:,i:(i+1)])
    vart = ((y-y_predt-y_pred)).var()
    vardat[-1,:] = (idcs[0],vart)

    result = {}

    
    #hist(y-y_predt-y_pred,41)

    #print(vardat)
    vsa = 1-(vardat[0,1])/var0
    #print(labels[int(vardat[0,0])],vsa*100,vsa)
    result[labels[int(vardat[0,0])]] = int(10000*(vsa))/10000
    for i in range(1,vardat.shape[0]):
        #print(vardat[i-1,1],vardat[i,1])
        if relative:
            var0 = vardat[i-1,1]
            vs = 1-(vardat[i,1])/var0
            result[labels[int(vardat[i,0])]]=int(10000*(vs))/10000
        else:
            vs = 1-(vardat[i,1])/var0
            result[labels[int(vardat[i,0])]]=int(10000*(vs-vsa))/10000
            vsa = vs
    return(result)

In [13]:
print(LinReg_varexplain_B(pddataT,mesT[:,1]-mesT[:,2]))
print(LinReg_varexplain_F(pddataT,mesT[:,1]-mesT[:,2]))
print(LinReg_varexplain_F(pddataT,mesT[:,1]-mesT[:,2],relative = True))

{'Rneck': 0.2598, 'A1': 0.3477, 'Dss': 0.0007, 'SA': 0.0004, 'variance': 20.8224}
{'Rneck': 0.2598, 'A1': 0.3246, 'SA': 0.0002, 'Dss': 0.0}
{'Rneck': 0.2598, 'A1': 0.4385, 'SA': 0.0006, 'Dss': 0.0}


In [14]:
print(LinReg_varexplain_B(pddataT,mesT[:,3]))
print(LinReg_varexplain_F(pddataT,mesT[:,3]))
print(LinReg_varexplain_F(pddataT,mesT[:,3],relative = True))

{'A1': 0.8907, 'Dss': 0.064, 'Rneck': 0.0004, 'SA': 0.0001, 'variance': 0.2015}
{'A1': 0.8907, 'Dss': 0.064, 'Rneck': 0.0004, 'SA': 0.0001}
{'A1': 0.8907, 'Dss': 0.5857, 'Rneck': 0.0105, 'SA': 0.004}


In [15]:
dataT.keys()

dict_keys(['#Sp', '#Bch', 'Dsb', 'Dss', 'd_interSp', 'Vtot', 'Vh', 'Ln', 'SA', 'nPSD', 'A1', 'A2', 'GPHN_SNR', 'DiS-EMchk', 'Lneck', 'maxDneck', 'minDneck', 'meanDneck', 'Rneck', 'Lhead', 'maxDhead', 'minDhead', 'meanDhead', 'Ah', 'Rhead', 'meanDneck_trunc70pc', 'L', 'D', 'AhA0', 'Rmorph'])

In [16]:
with open(folder+"dis_baselinev3_1000_80_65sh.pickle","rb") as f:
    vavgDiS,mesDiS,vtracsDiS,vtracsDDiS,vtracsSDiS,CtracsDiS = pickle.load(f)

with open(folder+"All_baseline_datasetv3_1000_80_65sh.pickle","rb") as f:
    vavgT,mesT,vtracsT,vtracsDT,vtracsST,CtracsT,me2T,_,dataT = pickle.load(f)

labels = ['Vh','A1','Ah','Dss','Rneck','Lhead']#,'meanDhead']
labels = ['A1','Dss','Rneck']#,'meanDhead']
labelsmod = labels
datt = dataT[labels[0]]
for l in labels[1:]:
    datt = column_stack((datt,dataT[l]))

#    datt = column_stack((dataT['Vh'],dataT['A1'],dataT['Ah'],dataT['Dss'],dataT['Rneck'],dataT['Lhead'],dataT['meanDhead']))

#labels = ('A1','Dss','Rneck','SA')
#datt = column_stack((dataT['A1'],dataT['Dss'],dataT['Rneck'],dataT['SA']))

pddataT = pd.DataFrame(data=datt,columns=labels)

print('*'*10+'V In Soma'+10*'*')
print(LinReg_varexplain_B(pddataT,mesT[:,3]))
print(LinReg_varexplain_F(pddataT,mesT[:,3]))
print(LinReg_varexplain_F(pddataT,mesT[:,3],relative = True))
print('*'*10+'V In Spine head'+10*'*')
print(LinReg_varexplain_B(pddataT,mesT[:,1]))
print(LinReg_varexplain_F(pddataT,mesT[:,1]))
print(LinReg_varexplain_F(pddataT,mesT[:,1],relative = True))
print('*'*10+'V sp-dend'+10*'*')
print(LinReg_varexplain_B(pddataT,mesT[:,1]-mesT[:,2]))
print(LinReg_varexplain_F(pddataT,mesT[:,1]-mesT[:,2]))
print(LinReg_varexplain_F(pddataT,mesT[:,1]-mesT[:,2],relative = True))
print('*'*10+'C In Spine head'+10*'*')
print(LinReg_varexplain_B(pddataT,mesT[:,4]))
print(LinReg_varexplain_F(pddataT,mesT[:,4]))
print(LinReg_varexplain_F(pddataT,mesT[:,4],relative = True))


#print('*'*10+'V sp-soma'+10*'*')
#print(LinReg_varexplain_B(pddataT,mesT[:,1]-mesT[:,3]))
#print(LinReg_varexplain_F(pddataT,mesT[:,1]-mesT[:,3]))

sel = dataT['nPSD'] == 2.0

datt = dataT[labels[0]][sel]
for l in labels[1:]:
    datt = column_stack((datt,dataT[l][sel]))

#    datt = column_stack((dataT['Vh'],dataT['A1'],dataT['Ah'],dataT['Dss'],dataT['Rneck'],dataT['Lhead'],dataT['meanDhead']))

#labels = ('A1','Dss','Rneck','SA')
#datt = column_stack((dataT['A1'],dataT['Dss'],dataT['Rneck'],dataT['SA']))

pddataT = pd.DataFrame(data=datt,columns=labels)

print('\n#'+'-'*15+'DiS!')
print('*'*10+'V In Soma'+10*'*')
print(LinReg_varexplain_B(pddataT,mesT[sel,3]))
print(LinReg_varexplain_F(pddataT,mesT[sel,3]))
print(LinReg_varexplain_F(pddataT,mesT[sel,3],relative = True))

print('*'*10+'V In Spine head'+10*'*')
print(LinReg_varexplain_B(pddataT,mesT[sel,1]))
print(LinReg_varexplain_F(pddataT,mesT[sel,1]))
print(LinReg_varexplain_F(pddataT,mesT[sel,1],relative = True))
print('*'*10+'V sp-dend'+10*'*')
print(LinReg_varexplain_B(pddataT,mesT[sel,1]-mesT[sel,2]))
print(LinReg_varexplain_F(pddataT,mesT[sel,1]-mesT[sel,2]))
print(LinReg_varexplain_F(pddataT,mesT[sel,1]-mesT[sel,2],relative = True))

print('*'*10+'C In Spine head'+10*'*')
print(LinReg_varexplain_B(pddataT,mesT[sel,4]))
print(LinReg_varexplain_F(pddataT,mesT[sel,4]))
print(LinReg_varexplain_F(pddataT,mesT[sel,4],relative = True))


**********V In Soma**********
{'A1': 0.8907, 'Dss': 0.064, 'Rneck': 0.0004, 'variance': 0.2015}
{'A1': 0.8907, 'Dss': 0.064, 'Rneck': 0.0004}
{'A1': 0.8907, 'Dss': 0.5857, 'Rneck': 0.0105}
**********V In Spine head**********
{'A1': 0.602, 'Rneck': 0.2015, 'Dss': 0.0001, 'variance': 39.3462}
{'A1': 0.602, 'Rneck': 0.1881, 'Dss': 0.0003}
{'A1': 0.602, 'Rneck': 0.4727, 'Dss': 0.0018}
**********V sp-dend**********
{'Rneck': 0.2598, 'A1': 0.3477, 'Dss': 0.0007, 'variance': 20.8224}
{'Rneck': 0.2598, 'A1': 0.3246, 'Dss': 0.0}
{'Rneck': 0.2598, 'A1': 0.4385, 'Dss': 0.0}
**********C In Spine head**********
{'A1': 0.3023, 'Rneck': 0.094, 'Dss': 0.0001, 'variance': 0.8904}
{'A1': 0.3023, 'Rneck': 0.0877, 'Dss': 0.0}
{'A1': 0.3023, 'Rneck': 0.1258, 'Dss': 0.0}

#---------------DiS!
**********V In Soma**********
{'A1': 0.7668, 'Dss': 0.1656, 'Rneck': 0.0041, 'variance': 0.1644}
{'A1': 0.7668, 'Dss': 0.1466, 'Rneck': 0.0091}
{'A1': 0.7668, 'Dss': 0.6292, 'Rneck': 0.1054}
**********V In Spine head**

In [17]:
labels = ['maxDhead','Dss','Lneck','Lhead','meanDneck_trunc70pc', 'A1']
labelmorph = labels
datt = dataT[labels[0]]
for l in labels[1:]:
    datt = column_stack((datt,dataT[l]))

#    datt = column_stack((dataT['Vh'],dataT['A1'],dataT['Ah'],dataT['Dss'],dataT['Rneck'],dataT['Lhead'],dataT['meanDhead']))

#labels = ('A1','Dss','Rneck','SA')
#datt = column_stack((dataT['A1'],dataT['Dss'],dataT['Rneck'],dataT['SA']))

pddataT = pd.DataFrame(data=datt,columns=labels)

print('*'*10+'V In Soma'+10*'*')
print(LinReg_varexplain_B(pddataT,mesT[:,3]))
print(LinReg_varexplain_F(pddataT,mesT[:,3]))
print(LinReg_varexplain_F(pddataT,mesT[:,3],relative = True))
print('*'*10+'V In Spine head'+10*'*')
print(LinReg_varexplain_B(pddataT,mesT[:,1]))
print(LinReg_varexplain_F(pddataT,mesT[:,1]))
print(LinReg_varexplain_F(pddataT,mesT[:,1],relative = True))
print('*'*10+'V sp-dend'+10*'*')
print(LinReg_varexplain_B(pddataT,mesT[:,1]-mesT[:,2]))
print(LinReg_varexplain_F(pddataT,mesT[:,1]-mesT[:,2]))
print(LinReg_varexplain_F(pddataT,mesT[:,1]-mesT[:,2],relative = True))
print('*'*10+'C In Spine head'+10*'*')
print(LinReg_varexplain_B(pddataT,mesT[:,4]))
print(LinReg_varexplain_F(pddataT,mesT[:,4]))
print(LinReg_varexplain_F(pddataT,mesT[:,4],relative = True))


#print('*'*10+'V sp-soma'+10*'*')
#print(LinReg_varexplain_B(pddataT,mesT[:,1]-mesT[:,3]))
#print(LinReg_varexplain_F(pddataT,mesT[:,1]-mesT[:,3]))

sel = dataT['nPSD'] == 2.0

datt = dataT[labels[0]][sel]
for l in labels[1:]:
    datt = column_stack((datt,dataT[l][sel]))

#    datt = column_stack((dataT['Vh'],dataT['A1'],dataT['Ah'],dataT['Dss'],dataT['Rneck'],dataT['Lhead'],dataT['meanDhead']))

#labels = ('A1','Dss','Rneck','SA')
#datt = column_stack((dataT['A1'],dataT['Dss'],dataT['Rneck'],dataT['SA']))

pddataT = pd.DataFrame(data=datt,columns=labels)

print('\n#'+'-'*15+'DiS!')
print('*'*10+'V In Soma'+10*'*')
print(LinReg_varexplain_B(pddataT,mesT[sel,3]))
print(LinReg_varexplain_F(pddataT,mesT[sel,3]))
print(LinReg_varexplain_F(pddataT,mesT[sel,3],relative = True))

print('*'*10+'V In Spine head'+10*'*')
print(LinReg_varexplain_B(pddataT,mesT[sel,1]))
print(LinReg_varexplain_F(pddataT,mesT[sel,1]))
print(LinReg_varexplain_F(pddataT,mesT[sel,1],relative = True))
print('*'*10+'V sp-dend'+10*'*')
print(LinReg_varexplain_B(pddataT,mesT[sel,1]-mesT[sel,2]))
print(LinReg_varexplain_F(pddataT,mesT[sel,1]-mesT[sel,2]))
print(LinReg_varexplain_F(pddataT,mesT[sel,1]-mesT[sel,2],relative = True))

print('*'*10+'C In Spine head'+10*'*')
print(LinReg_varexplain_B(pddataT,mesT[sel,4]))
print(LinReg_varexplain_F(pddataT,mesT[sel,4]))
print(LinReg_varexplain_F(pddataT,mesT[sel,4],relative = True))


**********V In Soma**********
{'A1': 0.8907, 'Dss': 0.064, 'Lneck': 0.0003, 'Lhead': 0.0002, 'meanDneck_trunc70pc': 0.0, 'maxDhead': 0.0, 'variance': 0.2015}
{'A1': 0.8907, 'Dss': 0.064, 'Lneck': 0.0004, 'Lhead': 0.0001, 'meanDneck_trunc70pc': 0.0, 'maxDhead': 0.0}
{'A1': 0.8907, 'Dss': 0.5857, 'Lneck': 0.0089, 'Lhead': 0.0044, 'meanDneck_trunc70pc': 0.0015, 'maxDhead': 0.0}
**********V In Spine head**********
{'A1': 0.602, 'Lneck': 0.14, 'meanDneck_trunc70pc': 0.0487, 'maxDhead': 0.0049, 'Dss': 0.0011, 'Lhead': 0.0001, 'variance': 39.3462}
{'A1': 0.602, 'Lneck': 0.1376, 'meanDneck_trunc70pc': 0.0273, 'maxDhead': 0.014, 'Dss': 0.0035, 'Lhead': 0.0002}
{'A1': 0.602, 'Lneck': 0.3457, 'meanDneck_trunc70pc': 0.105, 'maxDhead': 0.0604, 'Dss': 0.0162, 'Lhead': 0.0009}
**********V sp-dend**********
{'Lneck': 0.2262, 'A1': 0.2555, 'meanDneck_trunc70pc': 0.1027, 'maxDhead': 0.0059, 'Lhead': 0.0003, 'Dss': 0.0, 'variance': 20.8224}
{'Lneck': 0.2262, 'A1': 0.2511, 'meanDneck_trunc70pc': 0.0847, '

In [18]:
from sklearn.linear_model import BayesianRidge
bayesreg = BayesianRidge()
def BayReg_varexplain_F(X,y,labels= None, relative = False):
    if labels is None:
        try: 
            labels = X.keys()
        except:
            labels = arange(X.shape[1])
            
    idcs = arange(X.shape[1]).tolist()
    vardat = zeros((X.shape[1],2))
    
    y_predt = y*0.0
    var0 = var(y)
    minvar = var0
    for k in range(X.shape[1]-1):
        varts = zeros(len(idcs))
        for iv,i in enumerate(idcs):
            bayesreg.fit(X.values[:,i:(i+1)], y-y_predt)
            y_pred = bayesreg.predict(X.values[:,i:(i+1)])
            vart = ((y-y_predt-y_pred)).var()
            varts[iv] = vart
        sel = argmin(varts)
        i = idcs[sel]
        bayesreg.fit(X.values[:,i:(i+1)], y-y_predt)
        y_pred = bayesreg.predict(X.values[:,i:(i+1)])
        y_predt += y_pred

        vardat[k,:] = (idcs[sel],varts[sel])
        idcs.pop(sel)

    i = idcs[0]
    bayesreg.fit(X.values[:,i:(i+1)], y-y_predt)
    y_pred = bayesreg.predict(X.values[:,i:(i+1)])
    vart = ((y-y_predt-y_pred)).var()
    vardat[-1,:] = (idcs[0],vart)

    result = {}

    
    #hist(y-y_predt-y_pred,41)

    #print(vardat)
    vsa = 1-(vardat[0,1])/var0
    #print(labels[int(vardat[0,0])],vsa*100,vsa)
    result[labels[int(vardat[0,0])]] = int(10000*(vsa))/1e4
    for i in range(1,vardat.shape[0]):
        #print(vardat[i-1,1],vardat[i,1])
        if relative:
            var0 = vardat[i-1,1]
            vs = 1-(vardat[i,1])/var0
            result[labels[int(vardat[i,0])]]=int(10000*(vs))/1e4
        else:
            vs = 1-(vardat[i,1])/var0
            result[labels[int(vardat[i,0])]]=int(10000*(vs-vsa))/1e4
            vsa = vs
    return(result)

In [19]:
labels = ['Vh','A1','Ah','Dss','Rneck','Lhead']#,'meanDhead']
labels = labelsmod #,'meanDhead']
datt = dataT[labels[0]]
for l in labels[1:]:
    datt = column_stack((datt,dataT[l]))

#    datt = column_stack((dataT['Vh'],dataT['A1'],dataT['Ah'],dataT['Dss'],dataT['Rneck'],dataT['Lhead'],dataT['meanDhead']))

#labels = ('A1','Dss','Rneck','SA')
#datt = column_stack((dataT['A1'],dataT['Dss'],dataT['Rneck'],dataT['SA']))

pddataT = pd.DataFrame(data=datt,columns=labels)

print('*'*10+'V In Soma'+10*'*')
print(BayReg_varexplain_F(pddataT,mesT[:,3]))
print(BayReg_varexplain_F(pddataT,mesT[:,3],relative = True))
print('*'*10+'V In Spine head'+10*'*')
print(BayReg_varexplain_F(pddataT,mesT[:,1]))
print(BayReg_varexplain_F(pddataT,mesT[:,1],relative = True))
print('*'*10+'V sp-dend'+10*'*')
print(BayReg_varexplain_F(pddataT,mesT[:,1]-mesT[:,2]))
print(BayReg_varexplain_F(pddataT,mesT[:,1]-mesT[:,2],relative = True))
print('*'*10+'C In Spine head'+10*'*')
print(BayReg_varexplain_F(pddataT,mesT[:,4]))
print(BayReg_varexplain_F(pddataT,mesT[:,4],relative = True))


#print('*'*10+'V sp-soma'+10*'*')
#print(BayReg_varexplain_B(pddataT,mesT[:,1]-mesT[:,3]))
#print(BayReg_varexplain_F(pddataT,mesT[:,1]-mesT[:,3]))

sel = dataT['nPSD'] == 2.0

datt = dataT[labels[0]][sel]
for l in labels[1:]:
    datt = column_stack((datt,dataT[l][sel]))

#    datt = column_stack((dataT['Vh'],dataT['A1'],dataT['Ah'],dataT['Dss'],dataT['Rneck'],dataT['Lhead'],dataT['meanDhead']))

#labels = ('A1','Dss','Rneck','SA')
#datt = column_stack((dataT['A1'],dataT['Dss'],dataT['Rneck'],dataT['SA']))

pddataT = pd.DataFrame(data=datt,columns=labels)

print('\n#'+'-'*15+'DiS!')
print('*'*10+'V In Soma'+10*'*')
print(BayReg_varexplain_F(pddataT,mesT[sel,3]))
print(BayReg_varexplain_F(pddataT,mesT[sel,3],relative = True))

print('*'*10+'V In Spine head'+10*'*')
print(BayReg_varexplain_F(pddataT,mesT[sel,1]))
print(BayReg_varexplain_F(pddataT,mesT[sel,1],relative = True))
print('*'*10+'V sp-dend'+10*'*')
print(BayReg_varexplain_F(pddataT,mesT[sel,1]-mesT[sel,2]))
print(BayReg_varexplain_F(pddataT,mesT[sel,1]-mesT[sel,2],relative = True))

print('*'*10+'C In Spine head'+10*'*')
print(BayReg_varexplain_F(pddataT,mesT[sel,4]))
print(BayReg_varexplain_F(pddataT,mesT[sel,4],relative = True))


**********V In Soma**********
{'A1': 0.8907, 'Dss': 0.064, 'Rneck': 0.0004}
{'A1': 0.8907, 'Dss': 0.5857, 'Rneck': 0.0106}
**********V In Spine head**********
{'A1': 0.602, 'Rneck': 0.1878, 'Dss': 0.0002}
{'A1': 0.602, 'Rneck': 0.472, 'Dss': 0.0009}
**********V sp-dend**********
{'Rneck': 0.2598, 'A1': 0.3235, 'Dss': 0.0}
{'Rneck': 0.2598, 'A1': 0.437, 'Dss': 0.0}
**********C In Spine head**********
{'A1': 0.3023, 'Rneck': 0.0872, 'Dss': 0.0}
{'A1': 0.3023, 'Rneck': 0.125, 'Dss': 0.0}

#---------------DiS!
**********V In Soma**********
{'A1': 0.7668, 'Dss': 0.1446, 'Rneck': 0.0095}
{'A1': 0.7668, 'Dss': 0.6201, 'Rneck': 0.1079}
**********V In Spine head**********
{'A1': 0.4678, 'Rneck': 0.373, 'Dss': 0.0096}
{'A1': 0.4678, 'Rneck': 0.701, 'Dss': 0.0603}
**********V sp-dend**********
{'Rneck': 0.5123, 'A1': 0.2402, 'Dss': 0.0004}
{'Rneck': 0.5123, 'A1': 0.4927, 'Dss': 0.0016}
**********C In Spine head**********
{'A1': 0.4508, 'Rneck': 0.0909, 'Dss': 0.0004}
{'A1': 0.4508, 'Rneck': 0.165

In [20]:
labels = labelmorph
datt = dataT[labels[0]]
for l in labels[1:]:
    datt = column_stack((datt,dataT[l]))

#    datt = column_stack((dataT['Vh'],dataT['A1'],dataT['Ah'],dataT['Dss'],dataT['Rneck'],dataT['Lhead'],dataT['meanDhead']))

#labels = ('A1','Dss','Rneck','SA')
#datt = column_stack((dataT['A1'],dataT['Dss'],dataT['Rneck'],dataT['SA']))

pddataT = pd.DataFrame(data=datt,columns=labels)

print('*'*10+'V In Soma'+10*'*')
print(BayReg_varexplain_F(pddataT,mesT[:,3]))
print(BayReg_varexplain_F(pddataT,mesT[:,3],relative = True))
print('*'*10+'V In Spine head'+10*'*')
print(BayReg_varexplain_F(pddataT,mesT[:,1]))
print(BayReg_varexplain_F(pddataT,mesT[:,1],relative = True))
print('*'*10+'V sp-dend'+10*'*')
print(BayReg_varexplain_F(pddataT,mesT[:,1]-mesT[:,2]))
print(BayReg_varexplain_F(pddataT,mesT[:,1]-mesT[:,2],relative = True))
print('*'*10+'C In Spine head'+10*'*')
print(BayReg_varexplain_F(pddataT,mesT[:,4]))
print(BayReg_varexplain_F(pddataT,mesT[:,4],relative = True))


#print('*'*10+'V sp-soma'+10*'*')
#print(BayReg_varexplain_B(pddataT,mesT[:,1]-mesT[:,3]))
#print(BayReg_varexplain_F(pddataT,mesT[:,1]-mesT[:,3]))

sel = dataT['nPSD'] == 2.0

datt = dataT[labels[0]][sel]
for l in labels[1:]:
    datt = column_stack((datt,dataT[l][sel]))

#    datt = column_stack((dataT['Vh'],dataT['A1'],dataT['Ah'],dataT['Dss'],dataT['Rneck'],dataT['Lhead'],dataT['meanDhead']))

#labels = ('A1','Dss','Rneck','SA')
#datt = column_stack((dataT['A1'],dataT['Dss'],dataT['Rneck'],dataT['SA']))

pddataT = pd.DataFrame(data=datt,columns=labels)

print('\n#'+'-'*15+'DiS!')
print('*'*10+'V In Soma'+10*'*')
print(BayReg_varexplain_F(pddataT,mesT[sel,3]))
print(BayReg_varexplain_F(pddataT,mesT[sel,3],relative = True))

print('*'*10+'V In Spine head'+10*'*')
print(BayReg_varexplain_F(pddataT,mesT[sel,1]))
print(BayReg_varexplain_F(pddataT,mesT[sel,1],relative = True))
print('*'*10+'V sp-dend'+10*'*')
print(BayReg_varexplain_F(pddataT,mesT[sel,1]-mesT[sel,2]))
print(BayReg_varexplain_F(pddataT,mesT[sel,1]-mesT[sel,2],relative = True))

print('*'*10+'C In Spine head'+10*'*')
print(BayReg_varexplain_F(pddataT,mesT[sel,4]))
print(BayReg_varexplain_F(pddataT,mesT[sel,4],relative = True))


**********V In Soma**********
{'A1': 0.8907, 'Dss': 0.064, 'Lneck': 0.0003, 'Lhead': 0.0001, 'meanDneck_trunc70pc': 0.0001, 'maxDhead': 0.0}
{'A1': 0.8907, 'Dss': 0.5857, 'Lneck': 0.0082, 'Lhead': 0.0028, 'meanDneck_trunc70pc': 0.0023, 'maxDhead': 0.0}
**********V In Spine head**********
{'A1': 0.602, 'Lneck': 0.1374, 'meanDneck_trunc70pc': 0.0274, 'maxDhead': 0.0137, 'Dss': 0.0035, 'Lhead': 0.0}
{'A1': 0.602, 'Lneck': 0.3454, 'meanDneck_trunc70pc': 0.1053, 'maxDhead': 0.059, 'Dss': 0.0161, 'Lhead': 0.0}
**********V sp-dend**********
{'Lneck': 0.2262, 'A1': 0.2505, 'meanDneck_trunc70pc': 0.0849, 'maxDhead': 0.0151, 'Dss': 0.0004, 'Lhead': 0.0}
{'Lneck': 0.2262, 'A1': 0.3238, 'meanDneck_trunc70pc': 0.1623, 'maxDhead': 0.0346, 'Dss': 0.001, 'Lhead': 0.0}
**********C In Spine head**********
{'A1': 0.3023, 'meanDneck_trunc70pc': 0.0616, 'Lneck': 0.0011, 'Lhead': 0.0008, 'maxDhead': 0.0006, 'Dss': 0.0}
{'A1': 0.3023, 'meanDneck_trunc70pc': 0.0883, 'Lneck': 0.0017, 'Lhead': 0.0012, 'maxDhead

In [21]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
model = Pipeline([('poly', PolynomialFeatures(degree=4)),
                  ('linear', LinearRegression(fit_intercept=False))])

def ModReg_varexplain_F(X,y,labels= None, relative = False):
    if labels is None:
        try: 
            labels = X.keys()
        except:
            labels = arange(X.shape[1])
            
    idcs = arange(X.shape[1]).tolist()
    vardat = zeros((X.shape[1],2))
    
    y_predt = y*0.0
    var0 = var(y)
    minvar = var0
    for k in range(X.shape[1]-1):
        varts = zeros(len(idcs))
        for iv,i in enumerate(idcs):
            model.fit(X.values[:,i:(i+1)], y-y_predt)
            y_pred = model.predict(X.values[:,i:(i+1)])
            vart = ((y-y_predt-y_pred)).var()
            varts[iv] = vart
        sel = argmin(varts)
        i = idcs[sel]
        model.fit(X.values[:,i:(i+1)], y-y_predt)
        y_pred = model.predict(X.values[:,i:(i+1)])
        y_predt += y_pred

        vardat[k,:] = (idcs[sel],varts[sel])
        idcs.pop(sel)

    i = idcs[0]
    model.fit(X.values[:,i:(i+1)], y-y_predt)
    y_pred = model.predict(X.values[:,i:(i+1)])
    vart = ((y-y_predt-y_pred)).var()
    vardat[-1,:] = (idcs[0],vart)

    result = {}

    
    #hist(y-y_predt-y_pred,41)

    #print(vardat)
    vsa = 1-(vardat[0,1])/var0
    #print(labels[int(vardat[0,0])],vsa*100,vsa)
    result[labels[int(vardat[0,0])]] = int(10000*(vsa))/1e4
    for i in range(1,vardat.shape[0]):
        #print(vardat[i-1,1],vardat[i,1])
        if relative:
            var0 = vardat[i-1,1]
            vs = 1-(vardat[i,1])/var0
            result[labels[int(vardat[i,0])]]=int(10000*(vs))/1e4
        else:
            vs = 1-(vardat[i,1])/var0
            result[labels[int(vardat[i,0])]]=int(10000*(vs-vsa))/1e4
            vsa = vs
    return(result)

In [22]:
labels = ['Vh','A1','Ah','Dss','Rneck','Lhead']#,'meanDhead']
labels = labelsmod #,'meanDhead']
datt = dataT[labels[0]]
for l in labels[1:]:
    datt = column_stack((datt,dataT[l]))

#    datt = column_stack((dataT['Vh'],dataT['A1'],dataT['Ah'],dataT['Dss'],dataT['Rneck'],dataT['Lhead'],dataT['meanDhead']))

#labels = ('A1','Dss','Rneck','SA')
#datt = column_stack((dataT['A1'],dataT['Dss'],dataT['Rneck'],dataT['SA']))

pddataT = pd.DataFrame(data=datt,columns=labels)

print('*'*10+'V In Soma'+10*'*')
print(ModReg_varexplain_F(pddataT,mesT[:,3]))
print(ModReg_varexplain_F(pddataT,mesT[:,3],relative = True))
print('*'*10+'V In Spine head'+10*'*')
print(ModReg_varexplain_F(pddataT,mesT[:,1]))
print(ModReg_varexplain_F(pddataT,mesT[:,1],relative = True))
print('*'*10+'V sp-dend'+10*'*')
print(ModReg_varexplain_F(pddataT,mesT[:,1]-mesT[:,2]))
print(ModReg_varexplain_F(pddataT,mesT[:,1]-mesT[:,2],relative = True))
print('*'*10+'C In Spine head'+10*'*')
print(ModReg_varexplain_F(pddataT,mesT[:,4]))
print(ModReg_varexplain_F(pddataT,mesT[:,4],relative = True))


#print('*'*10+'V sp-soma'+10*'*')
#print(ModReg_varexplain_B(pddataT,mesT[:,1]-mesT[:,3]))
#print(ModReg_varexplain_F(pddataT,mesT[:,1]-mesT[:,3]))

sel = dataT['nPSD'] == 2.0

datt = dataT[labels[0]][sel]
for l in labels[1:]:
    datt = column_stack((datt,dataT[l][sel]))

#    datt = column_stack((dataT['Vh'],dataT['A1'],dataT['Ah'],dataT['Dss'],dataT['Rneck'],dataT['Lhead'],dataT['meanDhead']))

#labels = ('A1','Dss','Rneck','SA')
#datt = column_stack((dataT['A1'],dataT['Dss'],dataT['Rneck'],dataT['SA']))

pddataT = pd.DataFrame(data=datt,columns=labels)

print('\n#'+'-'*15+'DiS!')
print('*'*10+'V In Soma'+10*'*')
print(ModReg_varexplain_F(pddataT,mesT[sel,3]))
print(ModReg_varexplain_F(pddataT,mesT[sel,3],relative = True))

print('*'*10+'V In Spine head'+10*'*')
print(ModReg_varexplain_F(pddataT,mesT[sel,1]))
print(ModReg_varexplain_F(pddataT,mesT[sel,1],relative = True))
print('*'*10+'V sp-dend'+10*'*')
print(ModReg_varexplain_F(pddataT,mesT[sel,1]-mesT[sel,2]))
print(ModReg_varexplain_F(pddataT,mesT[sel,1]-mesT[sel,2],relative = True))

print('*'*10+'C In Spine head'+10*'*')
print(ModReg_varexplain_F(pddataT,mesT[sel,4]))
print(ModReg_varexplain_F(pddataT,mesT[sel,4],relative = True))


**********V In Soma**********
{'A1': 0.8935, 'Dss': 0.0655, 'Rneck': 0.0002}
{'A1': 0.8935, 'Dss': 0.6155, 'Rneck': 0.0052}
**********V In Spine head**********
{'A1': 0.6404, 'Rneck': 0.1942, 'Dss': 0.0005}
{'A1': 0.6404, 'Rneck': 0.5402, 'Dss': 0.0031}
**********V sp-dend**********
{'Rneck': 0.279, 'A1': 0.4094, 'Dss': 0.0014}
{'Rneck': 0.279, 'A1': 0.5678, 'Dss': 0.0046}
**********C In Spine head**********
{'A1': 0.5124, 'Rneck': 0.0537, 'Dss': 0.0025}
{'A1': 0.5124, 'Rneck': 0.1101, 'Dss': 0.0057}

#---------------DiS!
**********V In Soma**********
{'A1': 0.8103, 'Dss': 0.1365, 'Rneck': 0.0048}
{'A1': 0.8103, 'Dss': 0.7204, 'Rneck': 0.0921}
**********V In Spine head**********
{'A1': 0.5684, 'Rneck': 0.3097, 'Dss': 0.0163}
{'A1': 0.5684, 'Rneck': 0.7177, 'Dss': 0.1345}
**********V sp-dend**********
{'Rneck': 0.6002, 'A1': 0.2573, 'Dss': 0.0104}
{'Rneck': 0.6002, 'A1': 0.6436, 'Dss': 0.0733}
**********C In Spine head**********
{'A1': 0.6641, 'Rneck': 0.0435, 'Dss': 0.0092}
{'A1': 0.66

In [23]:
labels = ['Vh','A1','Ah','Dss','Rneck','Lhead']#,'meanDhead']
labels = labelmorph
datt = dataT[labels[0]]
for l in labels[1:]:
    datt = column_stack((datt,dataT[l]))

#    datt = column_stack((dataT['Vh'],dataT['A1'],dataT['Ah'],dataT['Dss'],dataT['Rneck'],dataT['Lhead'],dataT['meanDhead']))

#labels = ('A1','Dss','Rneck','SA')
#datt = column_stack((dataT['A1'],dataT['Dss'],dataT['Rneck'],dataT['SA']))

pddataT = pd.DataFrame(data=datt,columns=labels)

print('*'*10+'V In Soma'+10*'*')
print(ModReg_varexplain_F(pddataT,mesT[:,3]))
print(ModReg_varexplain_F(pddataT,mesT[:,3],relative = True))
print('*'*10+'V In Spine head'+10*'*')
print(ModReg_varexplain_F(pddataT,mesT[:,1]))
print(ModReg_varexplain_F(pddataT,mesT[:,1],relative = True))
print('*'*10+'V sp-dend'+10*'*')
print(ModReg_varexplain_F(pddataT,mesT[:,1]-mesT[:,2]))
print(ModReg_varexplain_F(pddataT,mesT[:,1]-mesT[:,2],relative = True))
print('*'*10+'C In Spine head'+10*'*')
print(ModReg_varexplain_F(pddataT,mesT[:,4]))
print(ModReg_varexplain_F(pddataT,mesT[:,4],relative = True))


#print('*'*10+'V sp-soma'+10*'*')
#print(ModReg_varexplain_B(pddataT,mesT[:,1]-mesT[:,3]))
#print(ModReg_varexplain_F(pddataT,mesT[:,1]-mesT[:,3]))

sel = dataT['nPSD'] == 2.0

datt = dataT[labels[0]][sel]
for l in labels[1:]:
    datt = column_stack((datt,dataT[l][sel]))

#    datt = column_stack((dataT['Vh'],dataT['A1'],dataT['Ah'],dataT['Dss'],dataT['Rneck'],dataT['Lhead'],dataT['meanDhead']))

#labels = ('A1','Dss','Rneck','SA')
#datt = column_stack((dataT['A1'],dataT['Dss'],dataT['Rneck'],dataT['SA']))

pddataT = pd.DataFrame(data=datt,columns=labels)

print('\n#'+'-'*15+'DiS!')
print('*'*10+'V In Soma'+10*'*')
print(ModReg_varexplain_F(pddataT,mesT[sel,3]))
print(ModReg_varexplain_F(pddataT,mesT[sel,3],relative = True))

print('*'*10+'V In Spine head'+10*'*')
print(ModReg_varexplain_F(pddataT,mesT[sel,1]))
print(ModReg_varexplain_F(pddataT,mesT[sel,1],relative = True))
print('*'*10+'V sp-dend'+10*'*')
print(ModReg_varexplain_F(pddataT,mesT[sel,1]-mesT[sel,2]))
print(ModReg_varexplain_F(pddataT,mesT[sel,1]-mesT[sel,2],relative = True))

print('*'*10+'C In Spine head'+10*'*')
print(ModReg_varexplain_F(pddataT,mesT[sel,4]))
print(ModReg_varexplain_F(pddataT,mesT[sel,4],relative = True))


**********V In Soma**********
{'A1': 0.8935, 'Dss': 0.0655, 'meanDneck_trunc70pc': 0.0012, 'Lneck': 0.0007, 'Lhead': 0.0003, 'maxDhead': 0.0002}
{'A1': 0.8935, 'Dss': 0.6155, 'meanDneck_trunc70pc': 0.0317, 'Lneck': 0.02, 'Lhead': 0.0085, 'maxDhead': 0.0076}
**********V In Spine head**********
{'A1': 0.6404, 'Lneck': 0.1409, 'meanDneck_trunc70pc': 0.0292, 'maxDhead': 0.0194, 'Dss': 0.0043, 'Lhead': 0.001}
{'A1': 0.6404, 'Lneck': 0.3919, 'meanDneck_trunc70pc': 0.1338, 'maxDhead': 0.1029, 'Dss': 0.0255, 'Lhead': 0.0063}
**********V sp-dend**********
{'A1': 0.2703, 'Lneck': 0.2884, 'meanDneck_trunc70pc': 0.0552, 'maxDhead': 0.039, 'Dss': 0.0034, 'Lhead': 0.002}
{'A1': 0.2703, 'Lneck': 0.3952, 'meanDneck_trunc70pc': 0.1253, 'maxDhead': 0.101, 'Dss': 0.0099, 'Lhead': 0.0059}
**********C In Spine head**********
{'A1': 0.5124, 'meanDneck_trunc70pc': 0.0409, 'maxDhead': 0.0121, 'Lneck': 0.0074, 'Dss': 0.0027, 'Lhead': 0.0007}
{'A1': 0.5124, 'meanDneck_trunc70pc': 0.084, 'maxDhead': 0.0272, 'Lne

In [24]:
[1,3,4,5]+[2]

[1, 3, 4, 5, 2]

# Analysis timed E/I

In [25]:
with open(folder+"dis_ga_glu02v3_1000_80_65sh.pickle","rb") as f:
    dis,mes1,mes01 = pickle.load(f)

with open(folder+"dis_glu_ga02v3_1000_80_65sh.pickle","rb") as f:
    dis,mes,mes0,times,times0 = pickle.load(f)

labels = ['Vh','A1','Ah','Dss','Rneck','Lhead']#,'meanDhead']
labels = labelsmod+['A2']
datt = dis[labels[0]]
for l in labels[1:]:
    datt = column_stack((datt,dis[l]))

pddataT = pd.DataFrame(data=datt,columns=labels)

print("\nGABA-GLU (2ms)----------------------------------------\n")
print('*'*10+'V in spine head'+10*'*')
print(LinReg_varexplain_B(pddataT,mes1[:,1]/mes01[:,1]))
print(LinReg_varexplain_F(pddataT,mes1[:,1]/mes01[:,1]))
print(LinReg_varexplain_F(pddataT,mes1[:,1]/mes01[:,1],relative = True))

print('*'*10+'V in dend shaft'+10*'*')
print(LinReg_varexplain_B(pddataT,mes1[:,2]/mes01[:,2]))
print(LinReg_varexplain_F(pddataT,mes1[:,2]/mes01[:,2]))
print(LinReg_varexplain_F(pddataT,mes1[:,2]/mes01[:,2],relative = True))

print('*'*10+'Calcium'+10*'*')
print(LinReg_varexplain_B(pddataT,mes1[:,4]/mes01[:,4]))
print(LinReg_varexplain_F(pddataT,mes1[:,4]/mes01[:,4]))
print(LinReg_varexplain_F(pddataT,mes1[:,4]/mes01[:,4],relative = True))


print("\nGLU-GABA (-2ms)----------------------------------------\n")
#print('*'*10+'V in spine head'+10*'*')
#print(LinReg_varexplain_B(pddataT,mes[:,1]/mes0[:,1]))
#print(LinReg_varexplain_F(pddataT,mes[:,1]/mes0[:,1]))
#print('*'*10+'V in dend shaft'+10*'*')
#print(LinReg_varexplain_B(pddataT,mes[:,2]/mes0[:,2]))
#print(LinReg_varexplain_F(pddataT,mes[:,2]/mes0[:,2]))
#print('*'*10+'Calcium'+10*'*')
#print(LinReg_varexplain_B(pddataT,mes[:,4]/mes0[:,4]))
#print(LinReg_varexplain_F(pddataT,mes[:,4]/mes0[:,4]))
print('*'*10+'Time reduction'+10*'*')
print(LinReg_varexplain_B(pddataT,times/times0))
print(LinReg_varexplain_F(pddataT,times/times0))
# print('*'*10+'Time'+10*'*')
# print(LinReg_varexplain_B(pddataT,times0))
# print(LinReg_varexplain_F(pddataT,times0))
labels = labelmorph+['A2']

datt = dis[labels[0]]
for l in labels[1:]:
    datt = column_stack((datt,dis[l]))

pddataT = pd.DataFrame(data=datt,columns=labels)

print("\nGABA-GLU (2ms)----------------------------------------\n")
print('*'*10+'V in spine head'+10*'*')
print(LinReg_varexplain_B(pddataT,mes1[:,1]/mes01[:,1]))
print(LinReg_varexplain_F(pddataT,mes1[:,1]/mes01[:,1]))
print(LinReg_varexplain_F(pddataT,mes1[:,1]/mes01[:,1],relative = True))
print('*'*10+'V in dend shaft'+10*'*')
print(LinReg_varexplain_B(pddataT,mes1[:,2]/mes01[:,2]))
print(LinReg_varexplain_F(pddataT,mes1[:,2]/mes01[:,2]))
print(LinReg_varexplain_F(pddataT,mes1[:,2]/mes01[:,2],relative = True))
print('*'*10+'Calcium'+10*'*')
print(LinReg_varexplain_B(pddataT,mes1[:,4]/mes01[:,4]))
print(LinReg_varexplain_F(pddataT,mes1[:,4]/mes01[:,4]))
print(LinReg_varexplain_F(pddataT,mes1[:,4]/mes01[:,4],relative = True))

print("\nGLU-GABA (-2ms)----------------------------------------\n")
#print('*'*10+'V in spine head'+10*'*')
#print(LinReg_varexplain_B(pddataT,mes[:,1]/mes0[:,1]))
#print(LinReg_varexplain_F(pddataT,mes[:,1]/mes0[:,1]))
#print('*'*10+'V in dend shaft'+10*'*')
#print(LinReg_varexplain_B(pddataT,mes[:,2]/mes0[:,2]))
#print(LinReg_varexplain_F(pddataT,mes[:,2]/mes0[:,2]))
#print('*'*10+'Calcium'+10*'*')
#print(LinReg_varexplain_B(pddataT,mes[:,4]/mes0[:,4]))
#print(LinReg_varexplain_F(pddataT,mes[:,4]/mes0[:,4]))
print('*'*10+'Time reduction'+10*'*')
print(LinReg_varexplain_B(pddataT,times/times0))
print(LinReg_varexplain_F(pddataT,times/times0))
print(LinReg_varexplain_F(pddataT,times/times0,relative = True))
# print('*'*10+'Time'+10*'*')
# print(LinReg_varexplain_B(pddataT,times0))
# print(LinReg_varexplain_F(pddataT,times0))
# print(LinReg_varexplain_F(pddataT,times0,relative = True))



GABA-GLU (2ms)----------------------------------------

**********V in spine head**********
{'A1': 0.2434, 'A2': 0.2822, 'Rneck': 0.0037, 'Dss': 0.0, 'variance': 0.017}
{'A1': 0.2434, 'A2': 0.2572, 'Rneck': 0.008, 'Dss': 0.0015}
{'A1': 0.2434, 'A2': 0.3399, 'Rneck': 0.016, 'Dss': 0.0031}
**********V in dend shaft**********
{'A1': 0.2352, 'A2': 0.2348, 'Rneck': 0.0464, 'Dss': 0.0001, 'variance': 0.026}
{'A1': 0.2352, 'A2': 0.214, 'Rneck': 0.0554, 'Dss': 0.0}
{'A1': 0.2352, 'A2': 0.2798, 'Rneck': 0.1006, 'Dss': 0.0}
**********Calcium**********
{'A1': 0.2429, 'Rneck': 0.2703, 'A2': 0.111, 'Dss': 0.0188, 'variance': 0.011}
{'A1': 0.2429, 'Rneck': 0.2541, 'A2': 0.1212, 'Dss': 0.0222}
{'A1': 0.2429, 'Rneck': 0.3357, 'A2': 0.241, 'Dss': 0.0581}

GLU-GABA (-2ms)----------------------------------------

**********Time reduction**********
{'A2': 0.4453, 'Dss': 0.1791, 'Rneck': 0.1258, 'A1': 0.0786, 'variance': 0.0114}
{'A2': 0.4453, 'Dss': 0.1725, 'Rneck': 0.1048, 'A1': 0.0729}

GABA-GLU (2ms)-

In [27]:
mes[:,1]

array([ 9.22822649,  5.67380436,  7.2392395 , 14.75093212, 22.86785701,
       19.1841277 ,  1.61791975, 17.33542057, 17.21383308,  9.4977521 ,
        9.64179931, 10.41366849, 10.78958572, 13.39656944, 15.10071388,
       14.32515081, 10.23021544,  9.64027121, 23.6098914 ,  8.2454328 ,
       21.53717165, 19.08682006,  5.62675129, 15.7871763 , 10.95892852,
        8.52546996,  2.03107714,  7.63823728,  4.72256321, 41.31964152,
        9.05141267,  6.65212759,  7.61783577, 12.81640351,  5.35813203,
        2.65606105,  2.93937202])

In [28]:
from sklearn.feature_selection import SelectKBest,VarianceThreshold,SelectFromModel, chi2, f_regression

from sklearn.linear_model import LassoCV


In [29]:
varthr = VarianceThreshold()
X = StandardScaler().fit_transform(datt)

varthr.fit_transform(datt)
yrt = mesT[:,3]

X_kbest=SelectKBest(score_func=f_regression,k=4).fit_transform(pddataT,yrt)


ValueError: Found input variables with inconsistent numbers of samples: [37, 384]

In [ ]:
print(X_kbest)
pddataT.head()

In [ ]:
regressor = LinearRegression()  
regressor.fit(pddataT, yrt)
y_pred = regressor.predict(datt)

sfm = SelectFromModel(regressor, threshold=1e-1)
sfm.fit(pddataT,yrt)
n_features = sfm.transform(pddataT).shape[1]

In [ ]:
X_transform = sfm.transform(pddataT)
print(n_features)
plt.title(
    " "
    "threshold %0.3f." % sfm.threshold)
feature1 = X_transform[:, 0]
feature2 = X_transform[:, 1]
scatter(feature1, feature2, c = yrt,alpha=0.8)
colorbar()
plt.xlabel("Feature number 1")
plt.ylabel("Feature number 2")
plt.ylim([np.min(feature2), np.max(feature2)])


In [ ]:
pddataT.corr()

In [ ]:
labels,varthr.variances_/varthr.variances_.sum()

In [ ]:
i = 1
regressor.fit(datt[:,i:(i+1)], mesT[:,3])
y_pred0 = regressor.predict(datt[:,i:(i+1)])
var0 = ((mesT[:,3]-y_pred0)).var()
print(var0)
i = 3
regressor.fit(datt[:,i:(i+1)], mesT[:,3]-y_pred0)
y_pred1 = regressor.predict(datt[:,i:(i+1)])
var1 = ((mesT[:,3]-y_pred0-y_pred1)).var()
y_predt = y_pred0+y_pred1
for i in [0,2,4,5,6]:
    regressor.fit(datt[:,i:(i+1)], mesT[:,3]-y_predt)
    y_pred2 = regressor.predict(datt[:,i:(i+1)])
    var2 = ((mesT[:,3]-y_predt-y_pred2)).var()
    print(labels[i],var2,var(mesT[:,3]))

In [ ]:
# #############################################################################
# Generate random sample data
centers = [[1, 1,5,2], [-1, -1,3,4], [1, -1,2,4]]
X, _ = make_blobs(n_samples=750, centers=centers, cluster_std=0.4,
                            random_state=0)

# Change X to real statistical data for spines
file = "spine_stats.dat"
X = load(file)


In [ ]:
X = StandardScaler().fit_transform(X)
X.shape

In [ ]:
# #############################################################################
# Compute DBSCAN
db = DBSCAN(eps=0.3, min_samples=10).fit(X)
core_samples_mask = zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

print('Estimated number of clusters: %d' % n_clusters_)
print('Estimated number of noise points: %d' % n_noise_)
#print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels_true, labels))
#print("Completeness: %0.3f" % metrics.completeness_score(labels_true, labels))
#print("V-measure: %0.3f" % metrics.v_measure_score(labels_true, labels))
#print("Adjusted Rand Index: %0.3f"
#      % metrics.adjusted_rand_score(labels_true, labels))
#print("Adjusted Mutual Information: %0.3f"
#      % metrics.adjusted_mutual_info_score(labels_true, labels))
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(X, labels))

In [ ]:
# #############################################################################
# Plot result
from matplotlib.pylab import cm
# Black removed and is used for noise instead.
unique_labels = set(labels)
colors = [cm.Spectral(each) for each in linspace(0, 1, len(unique_labels))]
for k, col in zip(unique_labels, colors):
    if k == -1:
        # Black used for noise.
        col = [0, 0, 0, 1]

    class_member_mask = (labels == k)

    xy = X[class_member_mask & core_samples_mask]
    plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
             markeredgecolor='k', markersize=14)

    xy = X[class_member_mask & ~core_samples_mask]
    plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
             markeredgecolor='k', markersize=6)

title('Estimated number of clusters: %d' % n_clusters_)
show()